<a href="https://colab.research.google.com/github/buganart/mocogan/blob/master/mocogan_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [2]:
#@markdown Check GPU, should be a Tesla V100
!nvidia-smi -L
import os
print(f"We have {os.cpu_count()} CPU cores.")

GPU 0: Tesla K80 (UUID: GPU-ba1c25a4-a5d4-f7fd-62eb-fd44bd700197)
We have 2 CPU cores.


In [3]:
#@markdown Mount google drive
from google.colab import drive
from google.colab import output
drive.mount('/content/drive')

from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    raise RuntimeError(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

def clear_on_success(msg="Ok!"):
    if _exit_code == 0:
        output.clear()
        print(msg)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Description


In [24]:
#@title Configuration

#@markdown Directories can be found via file explorer on the left by navigating into `drive` to the desired folders. 
#@markdown Then right-click and `Copy path`.
#@markdown - whether to generate videos from mocogan model
generate_mocogan = True #@param {type:"boolean"}
#@markdown - number of videos to generate
num_generate_video = 3 #@param {type:"integer"}
#@markdown - The output directory of generated videos 
video_dir = "/content/drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-mocogan" #@param {type:"string"}
#@markdown - set wandb run id of logged run to resume from there
resume_id = "3ald18we" #@param {type:"string"}

if not generate_mocogan:
    num_generate_video = 0
if resume_id:
    video_dir = Path(video_dir) / resume_id
    video_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
#@title Clone mocogan repo + Install Dependencies (wandb and log in)

!git clone https://github.com/buganart/mocogan

%cd /content/mocogan
# %pip install -r requirements.txt
%pip install sk-video
%pip install wandb==0.10.12

import wandb
from pathlib import Path
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")
clear_on_success("Dependencies installed.")

In [ ]:
#@title GENERATE
os.environ["WANDB_MODE"] = "dryrun"

!python generate.py \
--video_dir "$video_dir" \
--resume_id "$resume_id" \
--num_generate_video "$num_generate_video" 

In [ ]:
#@title MERGE VIDEO
#@markdown - **PLEASE DOUBLE CHECK THE video_blend_dir BELOW IS CORRECT.**
#@markdown - The input directory of source videos and output directory of blended videos
#@markdown - If the video_blend_dir is **empty**, **video_blend_dir = video_dir** above
video_blend_dir = "" #@param {type:"string"}
#@markdown  -  if generate_mocogan=False, then assume video_blend_dir already has some video files.
blend_videos = True #@param {type:"boolean"}
#@markdown - The TOTAL number of blend videos to generate
num_blend_video = 3 #@param {type:"integer"}

if not video_blend_dir:
    video_blend_dir = video_dir
#reference: https://www.geeksforgeeks.org/blending-of-two-videos-using-python/

import numpy as np
import cv2
from pathlib import Path
import skvideo.io

def combine_2videos(path_1, path_2):
    path_1 = Path(path_1)
    path_2 = Path(path_2)
    print("video1:", path_1.stem, "| video2:", path_2.stem)
    fg = cv2.VideoCapture(str(path_1))
    bg = cv2.VideoCapture(str(path_2))
    output_frames = []
    while True:
        ret, foreground = fg.read()
        _, background = bg.read()
        
        if ret:
            alpha = np.ones_like(foreground).astype(float) * 0.5
    
            # converting uint8 to float type
            foreground = foreground.astype(float)
            background = background.astype(float)
    
            # multiplying the foreground with alpha=0.5
            foreground = cv2.multiply(alpha, foreground)
    
            # multiplying the background with alpha=0.5
            background = cv2.multiply(alpha, background)
    
            # adding the masked foreground
            # and background together
            outImage = cv2.add(foreground,
                            background)
    
            # resizing the masked output
            # ims = cv2.resize(outImage, (96, 96))
            ims = outImage/255
            output_frames.append(ims)
    
            # showing the masked output video
            # cv2.imshow('Blended', ims)
    

        # if the actual video is over then there's
        # nothing in the foreground array thus
        # breaking from the while loop
        else:
            break
    #write video mp4
    output_frames = np.array(output_frames)* 255
    output_frames = output_frames.astype(np.uint8)
    savefilename = Path(path_1.parent) / ("combine_"+str(path_1.stem) +"-"+ str(path_2.stem) + ".mp4")
    skvideo.io.vwrite(savefilename, output_frames)

    print('Video Blending is done perfectly')


video_filelist = Path(video_blend_dir).rglob("*.mp4")
video_filelist = list(video_filelist)
for _ in range(num_blend_video):
    selected_indices = np.random.choice(len(video_filelist), size=2, replace=False)
    path_1 = video_filelist[selected_indices[0]]
    path_2 = video_filelist[selected_indices[1]]
    combine_2videos(path_1, path_2)